# **Parsing PDF stuff**

TBD content to use as PoC
- TBD one schematic publicly accessible - note the ECAD SW that generated it!
- TBD one other publicly accessible PDF content: table, sch in datasheet, datasheet itself...

In [ ]:
pdf_eg1 = './some.pdf'
pdf_eg2 = './some_other.pdf'

Idea of processing scenarios that will be explored with the help of this util
1) Text parsing/extraction (incl non-horizontal)
2) Ctrl+F in the text
3) Return the page nb where text was found
4) Print the page
5) OCR on the printed page with
- zoomin around the "frame" of the component.
- If no border, set arbitrary good-enough dim like 800x1000 (iterate tests)
- test how OCR recognize the "title" of the sch section (because e.g font is bigger)

***
### **E.g TODO** - attempt with `pdf2docx` lib
Goal: See if parsing directly .pdf => .docx renders such as "image in the background" with text around

In [ ]:
from pdf2docx import Converter

docx_file = 'test_output.docx'
nb_of_pages = 56 # fake data

# convert pdf to docx
cv = Converter(pdf_eg1) # pdf_eg2
cv.convert(docx_file, pages=[nb_of_pages]) #1st page has index 0
cv.close()

**Observations** 
- TBD

***
### **E.g TODO** - attempt with `pdf2docx` lib
Goal 1: Text parsing/extraction (incl. non-horizontal)

In [ ]:
import PyPDF2
from PyPDF2 import PdfReader

page_nb = 44 # fake data

reader = PdfReader(pdf_eg1) # pdf_eg2
number_of_pages = len(reader.pages)
page_eg = reader.pages[page_nb] #1st page has index 0
text_eg = page_eg.extract_text()

print(text_eg) #, end='\n'

# Instruction below returns general PDF file structure
page_eg

Goal 2: Image extraction in rectangles

In [ ]:
import svgwrite

reader_img = PdfReader(pdf_eg1) #pdf_eg2
page_eg_img = 44 # fake data

dwg = svgwrite.Drawing("some_drawing_to_save_for_test.svg", profile = "tiny")

# all below code is generic
def visitor_svg_rect(op, args, cm, tm):
    if op == b"re":
        (x, y, w, h) = (args[i].as_numeric() for i in range(4))
        dwg.add(dwg.rect((x, y), (w, h), stroke="red", fill_opacity=0.05))


def visitor_svg_text(text, cm, tm, fontDict, fontSize):
    (x, y) = (tm[4], tm[5])
    dwg.add(dwg.text(text, insert=(x, y), fill="blue"))


page_eg_img.extract_text(
    visitor_operand_before=visitor_svg_rect, visitor_text=visitor_svg_text
)
dwg.save()

Goal 3: Trying image extraction with .images attrib

In [ ]:
# same reader, page as above
count = 0

for image_file_object in page_eg_img.images:
    with open(str(count) + image_file_object.name, "wb") as fp:
        fp.write(image_file_object.data)
        count += 1

page_eg_img.images

***
### **E.g TODO** - attempt with `PyMuPDF + Pillow` lib
Goal 2+3: Image parsing/extraction

In [ ]:
import fitz # PyMuPDF
import io
from PIL import Image

pdf_eg1_opened = fitz.open(pdf_eg1)
# pdf_eg2_opened = fitz.open(pdf_eg2)

# iterate over PDF pages if multi-pages
for page_index in range(len(pdf_eg1_opened)):
    page = pdf_eg1_opened[page_index] # get page itself
    
    image_list = page.get_images()
    if image_list: # print nb of images found in page
        print(f"[+] Found a total of {len(image_list)} images in page {page_index}")
    else:
        print("[!] No images found on page", page_index)

    for image_index, img in enumerate(image_list, start=1):
        # get the XREF of the image
        xref = img[0]
        base_image = pdf_eg1_opened.extract_image(xref)

        image_bytes = base_image["image"] # extract the image bytes
        image_ext = base_image["ext"] # get the image extension
        image = Image.open(io.BytesIO(image_bytes)) # load it to PIL
        
        # save it to local disk
        image.save(open(f"image{page_index+1}_{image_index}.{image_ext}", "wb"))

***
### **E.g TODO** - attempt with `Camelot` lib
Goal 1: Text parsing/extraction (incl. non-horizontal)

In [ ]:
import camelot

tables = camelot.read_pdf(pdf_eg1)
print("Total tables extracted: ", tables.n)

#TODO solve Issue:
# "AttributeError: module 'camelot' has no attribute 'read_pdf' 
# --> maybe need java on the machine?"